In [ ]:
# Clone the repository of the official iTransformer github
!git clone https://github.com/thuml/iTransformer.git

Cloning into 'iTransformer'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 778 (delta 12), reused 7 (delta 7), pack-reused 760 (from 2)
Receiving objects: 100% (778/778), 11.76 MiB | 18.76 MiB/s, done.
Resolving deltas: 100% (471/471), done.


In [ ]:

# Install the required libraries in requirements.txt
!pip install -r iTransformer/requirements.txt

  Using cached pandas-1.5.3-cp39-cp39-macosx_11_0_arm64.whl (11.0 MB)
  Using cached scikit_learn-1.2.2-cp39-cp39-macosx_12_0_arm64.whl (8.5 MB)
  Using cached numpy-1.23.5-cp39-cp39-macosx_11_0_arm64.whl (13.4 MB)
  Using cached matplotlib-3.7.0-cp39-cp39-macosx_11_0_arm64.whl (7.3 MB)
  Using cached torch-2.0.0-cp39-none-macosx_11_0_arm64.whl (55.8 MB)
  Using cached reformer_pytorch-1.4.4-py3-none-any.whl (16 kB)
  Using cached einops-0.8.0-py3-none-any.whl (43 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit


Make Dataset file as required by original implementation of iTransformer 


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

asset = "BTC-USD"

# Period of test data (TODO: Change the period)
start_date = "2023-01-01"
end_date = "2025-01-01"

# Daily data of BTC-USD is used 

# auto_adjust=True is applied by default. 
# So,'Close' price is used instead of 'Adj Close'
df = yf.download(asset, start=start_date, end=end_date, interval='1d')['Close']
df.columns = ['Close']

# Features (TODO: Change the features)
df["Ret"] = df["Close"].pct_change()
df["Ret_5"] = df['Close'].pct_change(periods=5)    
df["Ret_10"] = df['Close'].pct_change(periods=10)    
df["Ret_15"] = df['Close'].pct_change(periods=15)
df["Ret_20"] = df['Close'].pct_change(periods=20)
df["Ret_25"] = df['Close'].pct_change(periods=25)
df["Ret_30"] = df['Close'].pct_change(periods=30)
df["Ret_40"] = df['Close'].pct_change(periods=40)
df["Ret_50"] = df['Close'].pct_change(periods=50)    

# Target variable (TODO: Change the target variable)
df["OT"] = df["Ret_25"].shift(-25)  

del df["Close"]
df = df.dropna()

df.reset_index(inplace=True)
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
df.rename(columns={'Date': 'date'}, inplace=True)

# Save the dataset as csv file
dataset_path = 'dataset_BTC_r25.csv'
df.to_csv(path_or_buf=dataset_path, index=False)

df.head()


[*********************100%***********************]  1 of 1 completed


,date,Ret,Ret_5,Ret_10,Ret_15,Ret_20,Ret_25,Ret_30,Ret_40,Ret_50,OT
0,2023-02-20 00:00:00,0.020615,0.021446,0.146780,0.081613,0.073030,0.077992,0.090067,0.384404,0.493475,0.104505
1,2023-02-21 00:00:00,-0.015820,0.034410,0.117301,0.073648,0.030037,0.058826,0.075524,0.295013,0.464266,0.103515
2,2023-02-22 00:00:00,-0.010129,-0.015337,0.110181,0.039741,0.030546,0.050269,0.054696,0.214935,0.450183,0.159157
3,2023-02-23 00:00:00,-0.009978,-0.028155,0.098101,0.043946,0.021245,0.007274,0.057916,0.141645,0.420100,0.159505
4,2023-02-24 00:00:00,-0.031292,-0.046429,0.043982,0.063206,-0.005731,0.015674,0.003472,0.110979,0.377828,0.214573


In [ ]:

# Set command line arguments for running the iTransformer
args = "--is_training 1 --model_id BTC_R25 --model iTransformer" 
args += " --data custom --freq d "
args += " --features M --seq_len 96 --pred_len 1"
args += " --e_layers 2 --enc_in 8 --dec_in 8 --c_out 8 --des 'Exp' --d_model 128 --d_ff 128" 
args += " --itr 1 --num_workers 1 --embed learned --inverse" 
args += " --root_path ./ --data_path " + dataset_path

# Run the iTransformer
!python iTransformer/run.py $args

Args in experiment:
Namespace(is_training=1, model_id='BTC_R25', model='iTransformer', data='custom', root_path='./', data_path='dataset_BTC_r25.csv', features='M', target='OT', freq='d', checkpoints='./checkpoints/', seq_len=96, label_len=48, pred_len=1, enc_in=8, dec_in=8, c_out=8, d_model=128, n_heads=8, e_layers=2, d_layers=1, d_ff=128, moving_avg=25, factor=1, distil=True, dropout=0.1, embed='learned', activation='gelu', output_attention=False, do_predict=False, num_workers=1, itr=1, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='MSE', lradj='type1', use_amp=False, use_gpu=False, gpu=0, use_multi_gpu=False, devices='0,1,2,3', exp_name='MTSF', channel_independence=False, inverse=True, class_strategy='projection', target_root_path='./data/electricity/', target_data_path='electricity.csv', efficient_training=False, use_norm=True, partial_start_index=0)
Use CPU
>>>>>>>start training : BTC_R25_iTransformer_custom_M_ft96_sl48_ll1_pl128_dm8_nh2_el1_dl1